<a href="https://colab.research.google.com/github/Benaziir/Practica5/blob/main/3_ProcesamientoDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procesamiento Datos Pandas

Para esta prueba deberá emplear los datos del archivo candidates.csv para lo cual deberá:

1. Almanecar los datos en una base de datos. (Asegurar el almacenamiento con el tipo de dato apropiado)
2. A partir de la lectura de los datos deberá generar un procesamiento reporte.
    
    (Cree el proceso que consideré más conveniente, realizarlo lo más complejo que le sea posible)
    
3. Realizar un código de envio de correos integrado en su solución.


El archivo candidates.csv contiene 50k filas de datos sobre candidatos. Los campos que estamos usando son:
- First Name
- Last Name
- Email
- Country
- Application Date
- Yoe (years of experience)
- Seniority
- Technology
- Code Challenge Score
- Technical Interview


,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/EvaluacionPC5/src/candidates.csv', delimiter=';')
# Crear una conexión a la base de datos SQLite (o crear una nueva si no existe)
conn = sqlite3.connect('candidates.db')
# Crear una tabla en la base de datos y definir los tipos de datos
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS candidates (
        FirstName TEXT,
        LastName TEXT,
        Email TEXT,
        Application_Date DATE,
        Country TEXT,
        YOE INTEGER,
        Seniority TEXT,
        Technology TEXT,
        Code_Challenge_Score INTEGER,
        Technical_Interview_Score INTEGER
    )
''')
# Almacenar los datos en la tabla
data.to_sql('candidates', conn, if_exists='replace', index=False)
# Cerrar la conexión a la base de datos
conn.close()
data.head()

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


In [17]:
# Análisis descriptivo de los datos
report = data.describe(include='all')
# Calcular estadísticas generales
total_candidatos = len(data)
promedio_experiencia = data['YOE'].mean()
max_skill_rating = data['Code Challenge Score'].max()
min_communication_rating = data['Technical Interview Score'].min()
# Calcular la cantidad de candidatos por nivel
candidatos_por_nivel = data['Seniority'].value_counts()
# Calcular la cantidad de candidatos por país
candidatos_por_pais = data['Country'].value_counts().head(10)  # Mostrar los 10 principales países
# Crear el informe
informe = f"""
--- Informe de Candidatos ---
Total de candidatos: {total_candidatos}
Promedio de Años de Experiencia: {promedio_experiencia:.2f} años
Máximo Skill Rating: {max_skill_rating}
Mínimo Communication Rating: {min_communication_rating}
Cantidad de candidatos por nivel:
{candidatos_por_nivel}
Top 10 países con más candidatos:
{candidatos_por_pais}
"""
# Guardar el informe en un archivo de texto
with open('informe_candidatos.txt', 'w') as file:
    file.write(informe)
# Mostrar el informe en la consola
print(informe)
print("Informe generado y guardado en 'informe_candidatos.txt'.")



--- Informe de Candidatos ---
Total de candidatos: 50000
Promedio de Años de Experiencia: 15.29 años
Máximo Skill Rating: 10
Mínimo Communication Rating: 0
Cantidad de candidatos por nivel:
Intern       7255
Mid-Level    7253
Trainee      7183
Junior       7100
Architect    7079
Lead         7071
Senior       7059
Name: Seniority, dtype: int64
Top 10 países con más candidatos:
Malawi                          242
Spain                           238
Svalbard & Jan Mayen Islands    234
Cook Islands                    234
Netherlands Antilles            234
Tajikistan                      233
Malaysia                        232
Samoa                           232
Italy                           232
Nauru                           231
Name: Country, dtype: int64

Informe generado y guardado en 'informe_candidatos.txt'.


In [19]:
smtp_server = 'smtp.gmail.com'  # Cambia esto al servidor SMTP que estés utilizando
smtp_port = 587  # Cambia esto al puerto adecuado
sender_email = 'benazircv@id.uff.br'
sender_password = '*****' #open('contrasena.txt').read().strip() #'tu_contraseña'

# Detalles del correo electrónico
receiver_email = 'benazcab775@gmail.com'
subject = 'Envio Reporte Candidates'
body = 'Adjunto lo solicitado'

# Crear el objeto MIMEMultipart
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))


# Adjuntar archivo
file_path = '/content/informe_candidatos.txt'  # Cambia la ruta al archivo que quieras adjuntar
with open(file_path, 'rb') as file:
    attachment = MIMEApplication(file.read(), _subtype="xlsx")
    attachment.add_header('Content-Disposition', 'attachment', filename=file_path)
    msg.attach(attachment)

# Iniciar la conexión con el servidor SMTP
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()  # Iniciar el modo seguro
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

print('Correo enviado exitosamente')

Correo enviado exitosamente
